In [1]:
import pandas as pd
import numpy as np
import re

# Time for some practice!

Time to put to practice what we have just covered. We need to start off by reading three csvs; naijaprop.csv, proppro.csv and trovit.csv. Save each of these dataframes as naijaprop, proppro and trovit respectively.

Print out the first 5 rows of each dataframe and descriptive information regarding the columns in each dataframe

In [2]:
#answer here

naijaprop = pd.read_csv('naijaprop.csv')
proppro = pd.read_csv('proppro.csv')
trovit = pd.read_csv('trovit.csv')

## Question 1

Create a new column for each dataframe, name this column <strong><i>source</i></strong> and label the values in the column based on the source of the listing. For example, all the listings in castles should be labeled as castles in the column we named <strong><i>source</i></strong>

In [3]:
#answer here

naijaprop['source'] = 'naijaprop'
proppro['source'] = 'proppop'
trovit['source'] = 'trovit'


## Question 2

Now, concatenate all the dataframes you have just read into one dataframe. This would be the equivalent of an outer merge, you should have a dataframe with <strong>18357 rows</strong> and <strong>6 columns</strong>, assign this to a variable named merged_df.

In [4]:
#answer here

merged_df = naijaprop.append([proppro, trovit]).reset_index(drop=True)
merged_df.shape
merged_df.columns


# Contrary to the suggestion above, I my merged_df
# DataFrame has 7 columns owing to the fact that a 
# new column source has been created and added to the Dataset

Index(['price', 'unit', 'location', 'bedrooms', 'baths', 'state', 'source'], dtype='object')

In [5]:
merged_df.state.isna().sum()

18337

In [6]:
merged_df.duplicated().sum()  #A high ammount of the data is duplicated

9152

## Question 3

We are going to do a bit of data cleaning, we need to remove the Naira sign from all the prices in our price column. This is a good add that for every issue you may encounter when writing code there is a good probability that someone has experienced something similar. If you do not know how to remove non numerical characters from a column, try Google this query with the phrase <strong><i>site:stackoverflow.com</i></strong> added to the end of your query and read a few answers with people trying different solutions and customize one of those for this question.

In [7]:
#answer here

merged_df['price'] = merged_df.price.apply(lambda x: x.replace('₦', '').replace(',', '') if pd.notna(x) else x).astype(np.float64)
merged_df.head()

,price,unit,location,bedrooms,baths,state,source
0,1000000.0,per Year,"15 - 17 Bolaji Balogun Street, Ayonusi Estate,...",5,6,Lagos,naijaprop
1,600000.0,per Year,"15 - 17 Bolaji Balogun Street, Ayonusi Estate,...",3,3,Lagos,naijaprop
2,250000.0,per Year,"badore,owode and addo roads,",1,1,Lagos,naijaprop
3,1000000.0,per Year,"ilupeju,",3,2,Lagos,naijaprop
4,600000.0,per Year,"badore road,",NaN,NaN,Lagos,naijaprop


## Question 4 

Find the missing and NaN values in the bedrooms and baths columns, how do you want to treat these missing values, there is no right or wrong answer. Find the most appropriate solution and explain your logic, ie why did you choose that particular method, what impact might it have on the accuracy and integrity of our data?

In [8]:
#answer here

merged_df[['bedrooms', 'baths']] = merged_df[['bedrooms', 'baths']].fillna(method='pad')
merged_df.head(10)

,price,unit,location,bedrooms,baths,state,source
0,1000000.0,per Year,"15 - 17 Bolaji Balogun Street, Ayonusi Estate,...",5,6,Lagos,naijaprop
1,600000.0,per Year,"15 - 17 Bolaji Balogun Street, Ayonusi Estate,...",3,3,Lagos,naijaprop
2,250000.0,per Year,"badore,owode and addo roads,",1,1,Lagos,naijaprop
3,1000000.0,per Year,"ilupeju,",3,2,Lagos,naijaprop
4,600000.0,per Year,"badore road,",3,2,Lagos,naijaprop
5,400000.0,per Year,"badore road,",1,1,Lagos,naijaprop
6,3700000.0,per Year,"Choba Uniport Road, Ph.,",2,2,Rivers,naijaprop
7,100.0,per Year,"Ozuoba NTA road Ph.,",2,2,Rivers,naijaprop
8,12500000.0,per Year,"Ikoyi,",3,3,Lagos,naijaprop
9,1500000.0,per Year,"alapere,",3,2,Lagos,naijaprop


## Question 5

You will notice that some listings do not have a state, instead the state is included in the location. Select all rows in the dataset where the value in the location columns contains the word <strong><i>State</i></strong> and where the state column contains an NaN value

In [9]:
#answer here

nan_df = merged_df[merged_df['state'].isna() & merged_df['location'].apply(lambda x: 'state' in x.lower().split())]
nan_df.head()

,price,unit,location,bedrooms,baths,state,source
3077,60000000.0,per Year,abiouwa estate lagos state alagomeji yaba lagos,4 bed,4 bath,NaN,proppop
3087,450000.0,per Year,"praisehill estate, arepo, ogun state arepo ogun",2 bed,2 bath,NaN,proppop
3098,50000000.0,per Year,at mende villa lagos state mende maryland lagos,4 bed,4 bath,NaN,proppop
4346,450000.0,per Year,"prince adebayo street, lakowe, ibeju lekki, l...",3 bed,2 bath,NaN,proppop
5042,800000.0,per Year,peace state ajao estate isolo lagos,2 bed,2 bath,NaN,proppop


## Question 6 

Ok, we are going to think of a way to clean up our data. Where we have an NaN value in the state column we are going to extract the state from the location column and append the value to the state column.
We are going to try and find a simple solution for this.

Let's start off by listing all unique states values in our merged_df dataframe

In [11]:
#answer here

unique_states = merged_df.state.unique()[:-1] # Get a list of the unique states in the dataset and remove the NaN element
unique_states = [i.lower() for i in unique_states]
unique_states

['lagos', 'rivers', 'ogun']

## Question 7

We are going to make the assumption that the list we just created is a list of all the states detailed in our list. Remember only about a third of listings have no state, we can assume that the remainder of the 2/3rd would probably have listings from all the states covered in our merged_df dataframe. Let's check how accurate our assumption is.

You need to find all rows under the column <strong><i>'location'</i></strong> containing the strings in the list we created in Question 6 and replace the NaN value in state with the name of the state that the row contains

For example if a row has the value <strong>Lagos State</strong> rename the state column with the value <strong>Lagos</strong>

Here is a tip, convert all the strings in location column to lower case and then search for strings that contain the values you found in Question 6 (convert these to lower case too)

In [12]:
merged_df.isna().sum()

price         288
unit            0
location        0
bedrooms        0
baths           0
state       18337
source          0
dtype: int64

In [13]:
merged_df['state'] = merged_df.apply(lambda x: ''.join([i.capitalize() for i in unique_states if i.lower() in x['location'].lower().replace(',', '').split()]) if any(item in unique_states for item in x['location'].lower().replace(',', '').split()) else x['state'], axis=1)
merged_df.head()

,price,unit,location,bedrooms,baths,state,source
0,1000000.0,per Year,"15 - 17 Bolaji Balogun Street, Ayonusi Estate,...",5,6,Lagos,naijaprop
1,600000.0,per Year,"15 - 17 Bolaji Balogun Street, Ayonusi Estate,...",3,3,Lagos,naijaprop
2,250000.0,per Year,"badore,owode and addo roads,",1,1,Lagos,naijaprop
3,1000000.0,per Year,"ilupeju,",3,2,Lagos,naijaprop
4,600000.0,per Year,"badore road,",3,2,Lagos,naijaprop


In [14]:
# merged_df['state'] = merged_df['state'].fillna(''.join([i for i in states if i.lower() in merged_df.state.lower().split()]))

In [15]:
# nan_df['state'] = nan_df.location.apply(lambda x: ''.join([i for i in unique_states if i.lower() in x.lower().split()]))

In [16]:
#answer here


# merged_df['state'] = merged_df.location.apply(lambda x: ', '.join([i for i in unique_states if i.lower() in x.lower().split()]) if merged_df.state[merged_df.location == x].any() else np.nan)



# merged_df['state'] = merged_df.location.apply(lambda x: ', '.join([i for i in unique_states if i.lower() in x.lower()]) if any([i for i in unique_states if i.lower() in x.lower()]) else np.nan)
# optionally try

# nan_df['state'] = nan_df.location.apply(lambda x: ', '.join([i for i in unique_states if i.lower() in x.lower()]) if x.upper().isin(unique_states) else np.nan)


# merged_df['state'] = merged_df['state'].com

## Question 8

Let' check how many NaN values we have under the state column now

In [17]:
#answer here

merged_df.state.isna().sum()

8466

## Question 9

Ok, so we still have NaN values, print out all the unique values in the <strong><i>location</i></strong> column for rows that still have an NaN values under the <strong><i>state</i></strong> column. You will notice that there actually aren't too many unique states. There are however, a lot of unique locations. Let's try and rename the states column based on the unique states we identify. Keep checking whether or not you have filled all NaN values under state.

<i>Please document your solution, explain what you did and why</i>

In [18]:
# first I identified all the possible states in the country in question which is nigeria  and save it in a variable called states

states = ['abia',
 'abuja',
 'adamawa',
 'akwa ibom',
 'anambra',
 'bauch',
 'bayelsa',
 'benue',
 'borno',
 'cross rivers',
 'delta',
 'ebonyi',
 'edo',
 'ekiti',
 'enugu',
 'gombe',
 'imo',
 'jigawa',
 'kaduna',
 'kano',
 'katsina',
 'kebbi',
 'kogi',
 'kwara',
 'nasarawa',
 'niger',
 'ondo',
 'oyo',
 'osun',
 'plateau',
 'sokoto',
 'taraba',
 'yobe',
 'zamfara']
len(states)

34

In [19]:
#answer here

# Search through the state column of the merged_df dataframe 
# if there exist any of the states collated in the states variable


merged_df['state'] = merged_df.apply(lambda x: ''.join([i.capitalize() for i in states if i.lower() in x['location'].lower().replace(',', '').split()]) if any(item in states for item in x['location'].lower().replace(',', '').split()) else x['state'], axis=1)


In [20]:
# This handles states labelled as Federal Capital Territory

merged_df['state'] = merged_df.apply(lambda x: 'Abuja' if 'Capital' in x['location'].replace(',', '').split() else x['state'], axis=1)


In [29]:
# This handles states with double lettered name such as cross rivers and akwa ibom



merged_df['state'] = merged_df.apply(lambda x: 'Cross Rivers' if 'cross river' in x['location'].replace(',', '').lower() else x['state'], axis=1)


merged_df['state'] = merged_df.apply(lambda x: 'Akwa Ibom' if 'akwa ibom' in x['location'].replace(',', '').lower() else x['state'], axis=1)


In [30]:
merged_df.state.isna().sum()

1588

In [31]:
merged_df.state.unique()

array(['Lagos', 'Rivers', 'Ogun', nan, 'Abuja', 'Oyo', 'Akwa Ibom',
       'Delta', 'Cross Rivers', 'Edo', 'Plateau', 'Imo', 'Osun',
       'EkitiEnugu', 'Abia', 'Kano', 'Enugu', 'Kaduna', 'Kwara',
       'Anambra', 'KanoNasarawa', 'AbujaOsun', 'AbujaKano', 'Ondo',
       'EkitiKwara', 'LagosOgun', 'Borno', 'Benue', 'Bayelsa', 'Adamawa',
       'Ekiti', 'OyoOsun', 'Sokoto'], dtype=object)

In [33]:
merged_df.loc[merged_df.state.isnull()].location.unique()

array([' '], dtype=object)

## Question 10 

Which state has the most expensive listing?

<i>Show the code you used to find this listing</i>

In [34]:
#answer here
most_expensive_state = merged_df['state'][merged_df.price == merged_df.price.max()].values
most_expensive_state

array(['Lagos'], dtype=object)

## Question 11

Which state has the least expensive listing?

<i>Show the code you used to find this listing</i>

In [35]:
#answer here#answer here
least_expensive_state = merged_df['state'][merged_df.price == merged_df.price.min()].values
least_expensive_state

array(['Lagos', 'Abuja', 'Abuja', 'Abuja', 'Abuja'], dtype=object)

## Question 12

Save <strong>merged_df</strong> as a csv file in your computer, use whatever name you will feel comfortable with

In [38]:
#answer here
merged_df.to_csv('merged_df.csv', index=False)

In [37]:
merged_df.to_csv?

In [36]:
merged_df

,price,unit,location,bedrooms,baths,state,source
0,1000000.0,per Year,"15 - 17 Bolaji Balogun Street, Ayonusi Estate,...",5,6,Lagos,naijaprop
1,600000.0,per Year,"15 - 17 Bolaji Balogun Street, Ayonusi Estate,...",3,3,Lagos,naijaprop
2,250000.0,per Year,"badore,owode and addo roads,",1,1,Lagos,naijaprop
3,1000000.0,per Year,"ilupeju,",3,2,Lagos,naijaprop
4,600000.0,per Year,"badore road,",3,2,Lagos,naijaprop
5,400000.0,per Year,"badore road,",1,1,Lagos,naijaprop
6,3700000.0,per Year,"Choba Uniport Road, Ph.,",2,2,Rivers,naijaprop
7,100.0,per Year,"Ozuoba NTA road Ph.,",2,2,Rivers,naijaprop
8,12500000.0,per Year,"Ikoyi,",3,3,Lagos,naijaprop
9,1500000.0,per Year,"alapere,",3,2,Lagos,naijaprop
